In [ ]:

import json
import os
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Assume the fine-tuned model and tokenizer have been saved.
model_path = "./fine_tuned_t5_model"
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)
model.eval() # Set the model to evaluation mode

feedback_log_file = "feedback_data.jsonl"
prefix = "grammar correction: "

def get_model_correction(text):
    """Uses the model to generate a correction."""
    inputs = tokenizer(prefix + text, return_tensors="pt")
    outputs = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=128
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def record_feedback(original_text, model_output, user_input):
    """Logs user feedback for a given correction."""
    with open(feedback_log_file, "a") as f:
        feedback_entry = {
            "original_text": original_text,
            "model_output": model_output,
            "user_input": user_input
        }
        f.write(json.dumps(feedback_entry) + "\n")
    print("Feedback recorded successfully.")

# --- Simulating the User Interface and Feedback Loop ---
def main_app_loop():
    print("--- Text Correction and Feedback Tool ---")
    print("Type 'exit' to quit.")
    print("When prompted, enter your preferred correction or type 'accept' to keep the model's suggestion.")
    
    while True:
        user_text = input("\nEnter your sentence: ")
        if user_text.lower() == 'exit':
            break

        model_suggestion = get_model_correction(user_text)
        print(f"Model Suggestion: {model_suggestion}")

        user_final_correction = input("Enter your final correction or 'accept': ")
        
        # Log the user's final decision
        if user_final_correction.lower() == 'accept':
            record_feedback(user_text, model_suggestion, model_suggestion)
        else:
            record_feedback(user_text, model_suggestion, user_final_correction)

if __name__ == "__main__":
    main_app_loop()

# --- Example of Retraining Prep ---
# Later, you could read the feedback log to create a new dataset
# for fine-tuning.
def create_retraining_dataset():
    retraining_data = {"incorrect": [], "correct": []}
    if os.path.exists(feedback_log_file):
        with open(feedback_log_file, "r") as f:
            for line in f:
                entry = json.loads(line)
                # Only use entries where the user's input was a valid correction
                if entry["user_input"] and entry["user_input"] != entry["original_text"]:
                    retraining_data["incorrect"].append(entry["original_text"])
                    retraining_data["correct"].append(entry["user_input"])
    
    if retraining_data["incorrect"]:
        print(f"\nCreated a new dataset for retraining with {len(retraining_data['incorrect'])} examples.")
        print("This new dataset can now be used to fine-tune the model again.")
    else:
        print("\nNo new feedback to create a retraining dataset.")
